In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_core.output_parsers import JsonOutputParser


parser = JsonOutputParser()
instructions = parser.get_format_instructions()
print(instructions)

Return a JSON object.


In [3]:
ai_response = "{\"이름\": \"홍길동\", \"나이\": 30}"
parsed_response = parser.parse(ai_response)
print(parsed_response)

{'이름': '홍길동', '나이': 30}


In [6]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

question = "가장 큰 대륙은?"
ai_response = "아시아입니다."
try:
    parsed_response = parser.parse(ai_response)
except Exception as e:
    print(f"파싱 오류: {e}")

파싱 오류: Invalid json output: 아시아입니다.
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 


In [14]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field, field_validator


# OpenAI 모델 설정
model = ChatOpenAI(model="gpt-4o", temperature=0.0)

# 원하는 데이터 구조 정의 (Pydantic v2 문법)
class FinancialAdvice(BaseModel):
    setup: str = Field(description="금융 조언 상황을 설정하기 위한 질문")
    advice: str = Field(description="질문을 해결하기 위한 금융 답변")

    # Pydantic v2 검증 로직
    @field_validator("setup", mode="before")
    @classmethod
    def setup_ends_with_question_mark(cls, v: str) -> str:
        if not v.endswith("?"):
            raise ValueError("잘못된 질문 형식입니다! 질문은 '?'로 끝나야 합니다.")
        return v

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 금융 전문가입니다. 사용자의 금융 질문에 전문적으로 답변하세요. 'setup' 필드는 반드시 질문 형태로 작성하고 끝을 '?'로 마무리하세요."),
    ("user", "질문: {query}")
])

# 모델에 structured output 바인딩 (자동 재시도 + 검증 포함)
model_with_structure = model.with_structured_output(FinancialAdvice)

# 체인 구성 (prompt | model | parser 대신 더 간단함)
chain = prompt | model_with_structure

result = chain.invoke({"query": "부동산 투자 시 금융 조언을 받을 수 있나요?"})
print(result)

setup='부동산 투자 시 금융 조언을 받을 수 있나요?' advice='부동산 투자는 장기적인 관점에서 자산을 증대시키는 데 효과적인 방법이 될 수 있습니다. 하지만, 투자 결정을 내리기 전에 몇 가지 중요한 요소를 고려해야 합니다.\n\n1. **시장 조사**: 투자하려는 지역의 부동산 시장 동향을 철저히 조사하세요. 지역 경제, 인구 증가, 인프라 개발 계획 등을 파악하는 것이 중요합니다.\n\n2. **재정 계획**: 투자에 필요한 자금을 어떻게 조달할 것인지 계획해야 합니다. 대출을 고려한다면, 이자율, 상환 기간, 월 상환액 등을 명확히 이해해야 합니다.\n\n3. **위험 관리**: 부동산 시장은 변동성이 있을 수 있습니다. 따라서, 투자 포트폴리오를 다양화하고, 비상시를 대비한 자금 마련 계획을 세우는 것이 중요합니다.\n\n4. **법률 및 세금 고려**: 부동산 투자와 관련된 법률 및 세금 문제를 이해하는 것이 중요합니다. 전문가의 도움을 받아 세금 혜택이나 의무를 명확히 파악하세요.\n\n5. **전문가 상담**: 부동산 중개인, 금융 전문가, 세무사 등과 상담하여 다양한 관점을 얻고, 보다 정보에 기반한 결정을 내리세요.\n\n이러한 요소들을 고려하여 신중하게 계획을 세운다면, 부동산 투자가 성공적인 자산 증대 수단이 될 수 있습니다.'
